In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver import Edge
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import os
import pywinauto
import win32api
from pywinauto.keyboard import send_keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from bs4 import BeautifulSoup
folder_path = "path_to_your_folder_path"
xpath_1='//*[@id="app"]/section/main/div/div[2]/div[1]/div[3]'
xpath_2='//*[@id="search-input-box"]/div[1]/textarea'
xpath_3='//*[@id="markdown-box-9"]'
xpath_4='//*[@id="assistant_id658a7988b8a9a98d38725745"]'
excel_path = 'path_to_your_excel_path.xlsx'
def extract_number(filename):
    for s in filename:
        if s.isdigit():
            start_index = filename.index(s)
            number_str = ""
            for char in filename[start_index:]:
                if char.isdigit():
                    number_str += char
                else:
                    break
            return int(number_str)
    return -1
def wait_for_element(xpath_expression):
    while True:
        try:
            element = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH, xpath_expression)))
            break
        except:
            pass
def wait_for_answer_text_change(driver,xpath):
    answer = driver.find_element(By.XPATH,xpath)
    previous_length = 0
    max_wait_time = 300 
    end_time = time.time() + max_wait_time 
    while time.time() < end_time:
        try:
            current_length = len(answer.text)
            if current_length == previous_length:
                break  
            previous_length = current_length
            time.sleep(5)
        except StaleElementReferenceException:
            answer = driver.find_element(By.XPATH,xpath)  
def LLM_answer(driver, XPATH, question):
    LLM_import = driver.find_element(By.XPATH, xpath_2)
    LLM_import.send_keys(question, Keys.ENTER)
    if __name__ == "__main__":
        wait_for_element(XPATH)
        wait_for_answer_text_change(driver, XPATH)
options=Options()
options.add_experimental_option('debuggerAddress','127.0.0.1:9222')
time.sleep(2)
driver=webdriver.Edge(options=options)
file_names = os.listdir(folder_path)
sorted_file_names = sorted(file_names, key=extract_number)
for file_name in sorted_file_names:
    if file_name.lower().endswith('.pdf'):
        file_path = os.path.join(folder_path, file_name)
        driver.find_element(By.XPATH,xpath_1).click()
        time.sleep(2) 
        app = pywinauto.Desktop()
        dlg = app["open"]
        dlg["Toolbar3"].click()
        send_keys(folder_path)
        send_keys("{VK_RETURN}")
        dlg["filename(&N):Edit"].type_keys(file_name)
        time.sleep(5)
        dlg["open(&O)"].click()
        time.sleep(30)  
        LLM_answer(driver, '//*[@id="markdown-box-0"]', 'prompt1')
        LLM_answer(driver, '//*[@id="markdown-box-1"]', 'prompt2')
        LLM_answer(driver, '//*[@id="markdown-box-2"]', 'prompt3')
        LLM_answer(driver, '//*[@id="markdown-box-3"]', 'prompt4')
        LLM_answer(driver, '//*[@id="markdown-box-4"]', 'prompt5')     
        LLM_answer(driver, '//*[@id="markdown-box-5"]', 'prompt6')
        LLM_answer(driver, '//*[@id="markdown-box-6"]', 'prompt7')
        LLM_answer(driver, '//*[@id="markdown-box-7"]', 'prompt8')
        LLM_answer(driver, '//*[@id="markdown-box-8"]', 'prompt9')
        LLM_answer(driver, '//*[@id="markdown-box-9"]', 'prompt10')
        table_html = driver.find_element(By.XPATH, '//*[@id="markdown-box-9"]').get_attribute("outerHTML")
        formatted_table_html = table_html.replace("<td>", "<td>|").replace("</td>", "|</td>")
        driver.execute_script(f"document.getElementById('markdown-box-9').outerHTML = '{formatted_table_html}'")
        markdown=driver.find_element(By.XPATH,xpath_3)
        markdown_text = markdown.text
        soup = BeautifulSoup(markdown_text, features="html.parser")
        table_markdown = soup.get_text()
        rows = [row for row in table_markdown.split("\n")[:] if "|" in row]
        processed_rows = []
        for row in rows:
            cells = [cell.strip() for cell in row.split("|") if cell.strip() != '']
            if cells:
                processed_rows.append(cells)
        for processed_row in processed_rows:
            print(processed_row)
        df = pd.DataFrame(processed_rows[:])
        book = load_workbook(excel_path)
        writer = pd.ExcelWriter(excel_path, engine='openpyxl') 
        writer.book = book
        last_row = writer.book.active.max_row
        for processed_row in dataframe_to_rows(df, index=False, header=True):
            writer.book.active.append(processed_row)
        writer.save()  
        driver.find_element(By.XPATH,xpath_4).click()
        time.sleep(2)